In [57]:
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import datetime
import pandas as pd


"""
Will cause all the output that normally goes to the R console,
to end up instead in a python list.
"""
# Import module #
import rpy2.rinterface_lib.callbacks
# Record output #
stdout = []
stderr = []
# Dummy functions #
def add_to_stdout(line): stdout.append(line)
def add_to_stderr(line): stderr.append(line)
# Keep the old functions #
stdout_orig = rpy2.rinterface_lib.callbacks.consolewrite_print
stderr_orig = rpy2.rinterface_lib.callbacks.consolewrite_warnerror
# Set the call backs #
rpy2.rinterface_lib.callbacks.consolewrite_print     = add_to_stdout
rpy2.rinterface_lib.callbacks.consolewrite_warnerror = add_to_stderr

    

In [58]:
# Initialization needed for all requests
utils = rpackages.importr("utils")
utils.install_packages("devtools")
devtools = rpackages.importr("devtools")
devtools.install_github("nwslR/nwslR")
nwslR = rpackages.importr("nwslR")

In [41]:
def _convert_and_fix_types(res):
    """Converts the raw output from rpy2 into Python types."""

    df = pandas2ri.rpy2py(res[0])
    types = (
        pandas2ri.rpy2py(res[1])
        if type(res[1]) == robjects.vectors.StrVector
        else [pandas2ri.rpy2py(x)[0] for x in res[1]]
    )
    for i, (col, t) in enumerate(zip(df.columns, types)):
        if "_pct" in col or "_accuracy" in col:
            df[col] = df[col].astype(float)
        elif t == "numeric":
            df[col] = df[col].astype("Int64")
        elif t == "logical":
            df[col] = df[col].astype(bool)
        elif t == "hms":
            df[col] = pd.to_datetime(df[col], unit="s").dt.time
        elif t == "Date":
            df[col] = pd.to_datetime(df[col], unit="d")
    return df

In [42]:
def _run_query(method, args = {}):
    
    return robjects.r(
        """
        data <- {method}({formatted_args})
        types <- sapply(data, class)
        list(data, types)
        """.format(
            method=method,
            formatted_args=",".join(list(map(lambda x: "'" + x[0] + "'='" + x[1] + "'", args.items())))
        )
    )

In [43]:
def load_matches():
    return _convert_and_fix_types(_run_query("load_matches"))


load_matches()

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 346.61TB/s
                                                                              
R[write to console]: 


,match_id,home_team_id,away_team_id,kickoff,time,stadium,city,year,season,home_team_score,away_team_score,home_penalty_score,away_penalty_score,won_on_penalites,last_updated
1,washington-spirit-vs-boston-breakers-2016-04-16,9,1,2016-04-16 19:00:00+00:00,19:00:00,None,None,2016,2016,1,0,0,0,False,2023-02-25
2,houston-dash-vs-chicago-red-stars-2016-04-16,4,2,2016-04-16 20:30:00+00:00,20:30:00,Shell Energy Stadium,"Houston, Texas",2016,2016,3,1,0,0,False,2023-02-25
3,fc-kansas-city-vs-western-new-york-flash-2016-...,3,10,2016-04-16 20:30:00+00:00,20:30:00,Children's Mercy Park,"Kansas City, Kansas",2016,2016,0,1,0,0,False,2023-02-25
4,seatle-reign-fc-vs-sky-blue-fc-2016-04-17,6,5,2016-04-17 19:00:00+00:00,19:00:00,UW Medicine Pitch at Memorial Stadium,Seattle,2016,2016,1,2,0,0,False,2023-02-25
5,portland-thorns-fc-vs-orlando-pride-2016-04-17,8,7,2016-04-17 22:00:00+00:00,22:00:00,Providence Park,"Portland, Oregon",2016,2016,2,1,0,0,False,2023-02-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,houston-dash-vs-kansas-city-current-2022-10-16,4,13,2022-10-16 17:00:00+00:00,17:00:00,Shell Energy Stadium,"Houston, Texas",2022,2022,1,2,0,0,False,2023-02-25
808,san-diego-wave-fc-vs-chicago-red-stars-2022-10-16,16,2,2022-10-16 22:00:00+00:00,22:00:00,Snapdragon Stadium,"San Diego, California",2022,2022,2,1,0,0,False,2023-02-25
809,portland-thorns-fc-vs-san-diego-wave-fc-2022-1...,8,16,2022-10-23 17:00:00+00:00,17:00:00,Providence Park,"Portland, Oregon",2022,2022,2,1,0,0,False,2023-02-25
810,ol-reign-vs-kansas-city-current-2022-10-23,6,13,2022-10-23 19:30:00+00:00,19:30:00,Lumen Field,"Seattle, Washington",2022,2022,0,2,0,0,False,2023-02-25


In [44]:
def load_players():
    return _convert_and_fix_types(_run_query("load_players"))

load_players()

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 278.01TB/s
                                                                              
R[write to console]: 


,player_id,player_first_name,player_last_name,player_short_first_name,player_short_last_name,player_match_name,player_nationality,player_position,player_place_of_birth,player_known_name,last_updated
1,1,Angela Marguerite,Salem,Angela,Salem,A. Salem,USA,Midfielder,Akron,None,2023-02-06
2,2,Louise,Schillgard,Louise,Schillgard,L. Schillgard,Sweden,Midfielder,Stockholm,None,2023-02-06
3,3,Natasha Khalila,Dowie,Natasha,Dowie,N. Dowie,England,Attacker,Abu Dhabi,None,2023-02-06
4,4,Kyah Pam,Simon,Kyah,Simon,K. Simon,Australia,Attacker,Blacktown,None,2023-02-06
5,5,Kylie Allyssa,Strom,Kylie,Strom,K. Strom,USA,Defender,Endicott,None,2023-02-06
...,...,...,...,...,...,...,...,...,...,...,...
732,732,Shuang,Wang,Shuang,Wang,Wang Shuang,China PR,Midfielder,Hubei,Wang Shuang,2023-02-06
733,733,Jaedyn Reese,Shaw,Jaedyn,Shaw,J. Shaw,USA,Midfielder,None,None,2023-02-06
734,734,Jackie,Altschuld,Jackie,Altschuld,J. Altschuld,USA,Midfielder,None,None,2023-02-06
735,735,Katie,Murray,Katie,Murray,K. Murray,USA,Midfielder,None,None,2023-02-06


In [45]:
def load_teams():
    return _convert_and_fix_types(_run_query("load_teams"))

load_teams()

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 438.41TB/s
                                                                              
R[write to console]: 


,team_id,team_name,team_name_short,team_club_name,team_abbreviation,first_season,last_season,last_updated
1,1,Boston Breakers,Boston Breakers,Boston Breakers,BOS,2016,2017,2023-02-18
2,2,Chicago Red Stars,Chicago Red Stars,Chicago Red Stars,CHI,2016,2022,2023-02-18
3,3,FC Kansas City,FC Kansas City,FC Kansas City,KC,2016,2017,2023-02-18
4,4,Houston Dash,Houston Dash,Houston Dash,HOU,2016,2022,2023-02-18
5,5,NJ/NY Gotham FC,NJ/NY Gotham FC,Gotham FC,NJY,2016,2022,2023-02-18
6,6,OL Reign,OL Reign,OL Reign,RGN,2016,2022,2023-02-18
7,7,Orlando Pride,Orlando Pride,Orlando Pride,ORL,2016,2022,2023-02-18
8,8,Portland Thorns FC,Portland Thorns FC,Portland Thorns,POR,2016,2022,2023-02-18
9,9,Washington Spirit,Washington Spirit,Washington Spirit,WAS,2016,2022,2023-02-18
10,10,Western New York Flash,Western New York Flash,Western New York Flash,WNY,2016,2016,2023-02-18


In [46]:
def load_metrics():
    return _convert_and_fix_types(_run_query("load_metrics"))

load_metrics()

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 416.14TB/s
                                                                              
R[write to console]: 


,metric,definition
1,accurate_back_zone_pass,A pass that is successful and ends in a player...
2,accurate_cross,Ball played from a wide position intending to ...
3,accurate_flick_on,Total number of passes which are “flicked” on ...
4,aerial_duels_lost,Unsuccessful aerial duels. Definition of an ae...
5,aerial_duels_won,Successful aerial duels
...,...,...
342,turnover,A player loses possession
343,was_fouled,Number of fouls on the player
344,woodwork_hit,Total number of occasions the ball hits the po...
345,yellow_cards,Total yellow cards


In [47]:
def load_team_season_stats(team_id, season):
    return _convert_and_fix_types(
        _run_query("load_team_season_stats", {"team_id": team_id, "season": season})
    )


load_team_season_stats("RGN", "2022")

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 468.38TB/s
                                                                              
R[write to console]: 


,team_id,season,tackle_last_man,tackles_last_man,possession_pct,long_passes_unsuccessful,lay_offs_unsuccessful,interceptions,open_play_crosses_unsuccessful,launches_successful,...,duels_lost,duels_won,shots_on_target,duels,penalties_conceded,blocks,aerial_duels_lost,goals_home,goals_away,last_updated
1,6,2022,1,1,50.0,725,16,280,289,120,...,1108,1226,136,2334,4,66,264,19,13,2023-02-18


In [48]:
def load_player_season_stats(team_id, season):
    return _convert_and_fix_types(
        _run_query("load_player_season_stats", {"team_id": team_id, "season": season})
    )


load_player_season_stats("RGN", "2022")

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 400.79TB/s
                                                                              
R[write to console]: 


,player_id,team_id,season,touches_total_opposition_box,launches_successful,duels_won,duels,goals_conceded_inside_box,tackles_total,passes_forward,...,saves_made,gk_smother,gk_distribution_unsuccessful,saves_made_inside_box,catches,drops,saves_made_outside_box,saves_made_parried,penalties_faced,last_updated
1,382,6,2022,13,2,84,150,9,35,253,...,0,0,0,0,0,0,0,0,0,2023-02-18
2,554,6,2022,16,1,30,61,3,20,36,...,0,0,0,0,0,0,0,0,0,2023-02-18
3,667,6,2022,7,0,9,12,2,2,24,...,0,0,0,0,0,0,0,0,0,2023-02-18
4,426,6,2022,77,6,123,221,11,34,136,...,0,0,0,0,0,0,0,0,0,2023-02-18
5,150,6,2022,3,22,71,131,14,50,491,...,0,0,0,0,0,0,0,0,0,2023-02-18
6,661,6,2022,0,0,0,1,0,0,3,...,0,0,0,0,0,0,0,0,0,2023-02-18
7,666,6,2022,2,0,2,8,2,1,8,...,0,0,0,0,0,0,0,0,0,2023-02-18
8,487,6,2022,15,17,64,109,14,35,410,...,0,0,0,0,0,0,0,0,0,2023-02-18
9,153,6,2022,76,2,84,158,13,24,248,...,0,0,0,0,0,0,0,0,0,2023-02-18
10,209,6,2022,9,0,6,15,1,3,22,...,0,0,0,0,0,0,0,0,0,2023-02-18


In [49]:
def load_team_match_stats(match_id):
    return  _convert_and_fix_types(_run_query("load_team_match_stats", {"match_id": match_id}))



load_team_match_stats("portland-thorns-fc-vs-kansas-city-current-2022-10-29")

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 426.47TB/s
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 356.75TB/s
                                                                              
R[write to console]: 


,match_id,team_id,period,entries_penalty_area,blocked_shots_inside_box,overrun,goals_forward,shots_corner,shots_off_target_inside_box,keypass_shot_on_target,...,goals_conceded_inside_box,error_lead_to_goal,saves,freekick_cross_accurate,goals_conceded,keeper_throws_accurate,saves_diving,shots_close_miss_high,saves_in_box,last_updated
1,portland-thorns-fc-vs-kansas-city-current-2022...,8,first_half,12,1,1,1,0,2,3,...,0,0,0,0,0,0,0,0,0,2023-02-26
2,portland-thorns-fc-vs-kansas-city-current-2022...,8,second_half,16,1,0,0,2,3,5,...,0,0,0,0,0,0,0,0,0,2023-02-26
3,portland-thorns-fc-vs-kansas-city-current-2022...,8,total,28,2,1,1,2,5,8,...,0,0,0,0,0,0,0,0,0,2023-02-26
4,portland-thorns-fc-vs-kansas-city-current-2022...,13,first_half,13,2,1,0,0,1,3,...,1,1,1,1,1,2,0,1,1,2023-02-26
5,portland-thorns-fc-vs-kansas-city-current-2022...,13,second_half,10,1,2,0,0,0,2,...,1,0,4,0,1,3,3,0,2,2023-02-26
6,portland-thorns-fc-vs-kansas-city-current-2022...,13,total,23,3,3,0,0,1,5,...,2,1,5,1,2,5,3,1,3,2023-02-26


In [50]:
def load_player_match_stats(match_id):
    return  _convert_and_fix_types(_run_query("load_player_match_stats", {"match_id": match_id}))



load_player_match_stats("portland-thorns-fc-vs-kansas-city-current-2022-10-29")

R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 253.88TB/s
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: indexed 0B in  0s, 0B/s
R[write to console]: 
R[write to console]: indexed 1.00TB in  0s, 476.63TB/s
                                                                              
R[write to console]: 


,match_id,team_id,player_id,touches,passes_leftside_opposition_half_total,passes_accurate,passes_final_third_total,launches_accurate,recoveries,entries_final_third,...,throws_keeper,offside_provoked,error_lead_to_goal,own_goals_conceded,freekick_cross,handballs,freekick_cross_accurate,shots_close_miss_high,shots_head_miss,last_updated
1,portland-thorns-fc-vs-kansas-city-current-2022...,8,364,27,3,12,2,2,8,2,...,0,0,0,0,0,0,0,0,0,2023-02-26
2,portland-thorns-fc-vs-kansas-city-current-2022...,8,205,72,2,30,16,1,10,8,...,0,0,0,0,0,0,0,0,0,2023-02-26
3,portland-thorns-fc-vs-kansas-city-current-2022...,8,77,57,9,43,6,2,13,6,...,0,0,0,0,0,0,0,0,0,2023-02-26
4,portland-thorns-fc-vs-kansas-city-current-2022...,8,309,52,9,30,6,1,10,4,...,0,0,0,0,0,0,0,0,0,2023-02-26
5,portland-thorns-fc-vs-kansas-city-current-2022...,8,568,80,9,29,14,2,12,6,...,0,0,0,0,0,0,0,0,0,2023-02-26
6,portland-thorns-fc-vs-kansas-city-current-2022...,8,119,41,6,27,10,0,9,6,...,0,0,0,0,0,0,0,0,0,2023-02-26
7,portland-thorns-fc-vs-kansas-city-current-2022...,8,676,59,7,36,16,0,10,8,...,0,0,0,0,0,0,0,0,0,2023-02-26
8,portland-thorns-fc-vs-kansas-city-current-2022...,8,207,39,3,20,9,0,6,3,...,0,0,0,0,0,0,0,0,0,2023-02-26
9,portland-thorns-fc-vs-kansas-city-current-2022...,8,495,37,1,12,18,0,3,3,...,0,0,0,0,0,0,0,0,0,2023-02-26
10,portland-thorns-fc-vs-kansas-city-current-2022...,8,496,60,4,17,12,0,2,4,...,0,0,0,0,0,0,0,0,0,2023-02-26


In [52]:
def available_teams():
    return robjects.r("available_teams()")

available_teams()

R[write to console]: Error in available_teams() : could not find function "available_teams"



RRuntimeError: Error in available_teams() : could not find function "available_teams"
